# Initialization

In [0]:
%run "/Workspace/Users/amberasad0299@gmail.com/databricks_data_lakehouse_project/scripts/silver/silver_util"

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col

## Read Bronze Table

In [0]:
bronze_table_name = "crm_prd_info" 
df = read_bronze(spark, bronze_table_name)

# Silver Transformations

## Trimming

In [0]:
df = trim_all_strings(df)

## Product Key Parsing

In [0]:
df = df.withColumn("cat_id", F.regexp_replace(F.substring(col("prd_key"), 1, 5), "-", "_"))
df = df.withColumn("prd_key", F.substring(col("prd_key"), 7, F.length(col("prd_key"))))

## Cost Cleanup

In [0]:
df = df.withColumn("prd_cost", F.coalesce(col("prd_cost"), F.lit(0)))

## Product Line Normalization

In [0]:
df = df.withColumn(
    "prd_line",
    F.when(F.upper(col("prd_line")) == "M", "Mountain")
     .when(F.upper(col("prd_line")) == "R", "Road")
     .when(F.upper(col("prd_line")) == "S", "Other Sales")
     .when(F.upper(col("prd_line")) == "T", "Touring")
     .otherwise("n/a")
)

## Date Casting

In [0]:
df = df.withColumn("prd_start_dt", col("prd_start_dt").cast(DateType()))

## Renaming Columns

In [0]:
RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}
df = rename_columns(df, RENAME_MAP)

## Sanity Check before Writing

In [0]:
display(df.limit(10))

# Writing Silver Table

In [0]:
silver_table_name = "crm_products" 
write_silver(df, silver_table_name)

## Sanity Check after Writing

In [0]:
%sql
SELECT * FROM workspace.silver.crm_products LIMIT 10